In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("flipkart_product_cleaned_final_sentiment.csv")

In [3]:
df.head()

,ProductName,Price,Rate,Review,Summary,sentiment
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,3999,1,super !,great cooler excel air flow price amaz unbelie...,0.650000
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,3999,1,awesom,best budget fit cooler nice cool,0.587500
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,3999,1,fair,qualiti good power air decent,0.433333
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,3999,0,useless product,bad product fan,-0.700000
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,3999,1,fair,ok ok product,0.500000


In [4]:
df.loc[df['Summary'].isna(), 'Summary'] = ''  # Replace NaN values in the Summary column with an empty string text using loc.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df['Summary'], df['Rate'], test_size=0.2, random_state=7)

In [7]:
# Convert text data to TF-IDF features for traditional ML models
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [11]:
from catboost import CatBoostClassifier, Pool

# Initialize CatBoost Classifier
model = CatBoostClassifier(
    iterations=1000,  # number of trees
    depth=6,          # tree depth
    learning_rate=0.1,
    loss_function='Logloss',  # binary classification loss
    verbose=100
)

# Training
model.fit(x_train_tfidf, y_train, eval_set=(x_test_tfidf, y_test), early_stopping_rounds=100)

0:	learn: 0.6063237	test: 0.6071910	best: 0.6071910 (0)	total: 439ms	remaining: 7m 18s
100:	learn: 0.2145780	test: 0.2211443	best: 0.2211443 (100)	total: 28.9s	remaining: 4m 17s
200:	learn: 0.1972528	test: 0.2067038	best: 0.2067038 (200)	total: 57s	remaining: 3m 46s
300:	learn: 0.1886200	test: 0.2006595	best: 0.2006595 (300)	total: 1m 24s	remaining: 3m 17s
400:	learn: 0.1828519	test: 0.1972191	best: 0.1972191 (400)	total: 1m 52s	remaining: 2m 48s
500:	learn: 0.1786418	test: 0.1948596	best: 0.1948596 (500)	total: 2m 20s	remaining: 2m 20s
600:	learn: 0.1750118	test: 0.1930144	best: 0.1930144 (600)	total: 2m 49s	remaining: 1m 52s
700:	learn: 0.1721509	test: 0.1917919	best: 0.1917805 (695)	total: 3m 17s	remaining: 1m 24s
800:	learn: 0.1703209	test: 0.1912972	best: 0.1912920 (798)	total: 3m 45s	remaining: 55.9s
900:	learn: 0.1681569	test: 0.1904635	best: 0.1904611 (893)	total: 4m 13s	remaining: 27.8s
999:	learn: 0.1662326	test: 0.1897930	best: 0.1897889 (998)	total: 4m 40s	remaining: 0us

b

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Predict the labels for the test set
y_pred = model.predict(x_test_tfidf)

# Calculate accuracy
accuracy=accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Print classification report
print(classification_report(y_test, y_pred))

# ROC AUC score
y_pred_proba = model.predict_proba(x_test_tfidf)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC Score: {roc_auc}")

Accuracy: 0.9290367542344635
              precision    recall  f1-score   support

           0       0.85      0.66      0.74      5134
           1       0.94      0.98      0.96     27869

    accuracy                           0.93     33003
   macro avg       0.89      0.82      0.85     33003
weighted avg       0.93      0.93      0.93     33003

ROC AUC Score: 0.9472528919352957


In [13]:
print(y_pred)

[1 1 1 ... 1 1 0]


In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# Predict on the test data
y_pred = model.predict(x_test_tfidf)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[ 3397  1737]
 [  605 27264]]


In [17]:
# Check class distribution in training data
train_class_distribution = y_train.value_counts(normalize=True)
print("Training Class Distribution:\n", train_class_distribution)

# Check class distribution in test data
test_class_distribution = y_test.value_counts(normalize=True)
print("Test Class Distribution:\n", test_class_distribution)

Training Class Distribution:
 1    0.851018
0    0.148982
Name: Rate, dtype: float64
Test Class Distribution:
 1    0.844438
0    0.155562
Name: Rate, dtype: float64
